# Step 3: Inference

## Connecting to Hopsworks Feature Store

In [ ]:
import hopsworks
from ipython_secrets import *

KEY = get_secret("HOPSWORKS_API_KEY")
project = hopsworks.login(host="c.app.hopsworks.ai", api_key_value=KEY)

fs = project.get_feature_store()

## Feature Retrieval

In [ ]:
trans_fg = fs.get_feature_group(
    name="transactions_fraud_streaming_fg_" + str(project.id),
    version=1,
)

In [ ]:
# Retrieve the first 5 unique credit card numbers (cc_nums)
cc_nums = trans_fg.select("cc_num").show(5).cc_num.values

# Display the obtained cc_nums
cc_nums

## Model Registry

In [ ]:
# Get the Model Registry
mr = project.get_model_registry()

## Fetch Deployment

In [ ]:
# Access the Model Serving
ms = project.get_model_serving()

# Specify the deployment name
deployment_name = "fraudonlinemodeldeployment"

# Get the deployment with the specified name
deployment = ms.get_deployment(deployment_name)

# Start the deployment and wait for it to be in a running state for up to 300 seconds
deployment.start(await_running=300)

## Predict using deployment

In [ ]:
# Get the first credit card number
cc_num = cc_nums[0]
cc_num

In [ ]:
# Make a prediction
deployment.predict(
    inputs=[cc_num],
)

In [ ]:
# Predict for several cc_nums
predictions = [
    deployment.predict(inputs=[int(cc_num)])["predictions"] for cc_num in cc_nums
]
predictions

## Stop Deployment

In [ ]:
# Stop the deployment
deployment.stop(await_stopped=180)